#### HTTPRouter using a Trie

[//]: # (Image References)


[image1]: image.jpg "image file"

For this exercise we are going to implement an HTTPRouter like you would find in a typical web server using the Trie data structure we learned previously.

There are many different implementations of HTTP Routers such as regular expressions or simple string matching, but the Trie is an excellent and very efficient data structure for this purpose.

The purpose of an HTTP Router is to take a URL path like "/", "/about", or "/blog/2019-01-15/my-awesome-blog-post" and figure out what content to return. In a dynamic web server, the content will often come from a block of code called a handler.

![alt text][image1]

First we need to implement a slightly different Trie than the one we used for autocomplete. Instead of simple words the Trie will contain a part of the http path at each node, building from the root node `/`

In addition to a path though, we need to know which function will handle the http request. In a real router we would probably pass an instance of a class like Python's [SimpleHTTPRequestHandler](https://docs.python.org/3/library/http.server.html#http.server.SimpleHTTPRequestHandler) which would be responsible for handling requests to that path. For the sake of simplicity we will just use a string that we can print out to ensure we got the right handler

We could split the path into letters similar to how we did the autocomplete Trie, but this would result in a Trie with a very large number of nodes and lengthy traversals if we have a lot of pages on our site. A more sensible way to split things would be on the parts of the path that are separated by slashes `("/")`. A Trie with a single path entry of: "/about/me" would look like:

`(root, None) -> ("about", None) -> ("me", "About Me handler")`

We can also simplify our RouteTrie a bit by excluding the suffixes method and the endOfWord property on RouteTrieNodes. We really just need to insert and find nodes, and if a RouteTrieNode is not a leaf node, it won't have a handler which is fine.

In [1]:
# A RouteTrie will store our routes and their associated handlers
class RouteTrie:
    def __init__(self, handler):
        # Initialize the trie with an root node and a handler, this is the root path or home page node
        self.root = RouteTrieNode()
        self.handler = handler
    
    def insert(self, path_list, handler):
        # Similar to our previous example you will want to recursively add nodes
        # Make sure you assign the handler to only the leaf (deepest) node of this path
        node = self.root
        for path in path_list:
            if path not in node.children:
                node.insert(path)
            node = node.children[path]
        node.handler = handler
        
        
        
    def find(self, path):
        # Starting at the root, navigate the Trie to find a match for this path
        # Return the handler for a match, or None for no match
        if self.root is None:
            return self.not_found
        node = self.root
        for part in path:
            if part not in node.children:
                return self.not_found
            node = node.children[part]
        return node.handler

    
# A RouteTrieNode will be similar to our autocomplete TrieNode... with one additional element, a handler.
class RouteTrieNode:
    def __init__(self, handler=None):
        # Initialize the node with children as before, plus a handler
        self.children = {}
        self.handler = handler
    def insert(self, path, handler=None):
        # Insert the node as before
        # Insert the node as before
        self.children[path] = RouteTrieNode(handler)        

Next we need to implement the actual Router. The router will initialize itself with a RouteTrie for holding routes and associated handlers. It should also support adding a handler by path and looking up a handler by path. All of these operations will be delegated to the RouteTrie.

Hint: the RouteTrie stores handlers under path parts, so remember to split your path around the '/' character

Bonus Points: Add a not found handler to your Router which is returned whenever a path is not found in the Trie.

More Bonus Points: Handle trailing slashes! A request for '/about' or '/about/' are probably looking for the same page. Requests for '' or '/' are probably looking for the root handler. Handle these edge cases in your Router.

In [2]:
# The Router class will wrap the Trie and handle 
class Router:
    def __init__(self, handler=None, not_found=None):
        # Create a new RouteTrie for holding our routes
        # You could also add a handler for 404 page not found responses as well!
        self.root = RouteTrie(handler)
        self.handler = handler
        self.not_found = not_found if not_found != "" or not_found is not None else "HTTP 404 Page Not Found"

    def add_handler(self, path, handler):
        # Add a handler for a path
        # You will need to split the path and pass the pass parts
        # as a list to the RouteTrie
        node = self.root
        path_list = self.split_path(path)
        node.insert(path_list, handler)

    def lookup(self, path):
        # lookup path (by parts) and return the associated handler
        # you can return None if it's not found or
        # return the "not found" handler if you added one
        # bonus points if a path works with and without a trailing slash
        # e.g. /about and /about/ both return the /about handler
        if path == '/':
            
            return "You Got Root!"
        
        node = self.root.root
        path_list = self.split_path(path)
        for part in path_list:
            if part not in node.children:
                return "HTTP 404 Page Not Found"
            node = node.children[part]
        if not node.handler:
            return "HTTP 404 Page Not Found"
        return node.handler

    def split_path(self, path):
        # you need to split the path into parts for 
        # both the add_handler and loopup functions,
        # so it should be placed in a function here
        
        
        path_list = path.split('/')

        # discard the empty strings
        if path_list[0] == '':
            path_list = path_list[1:]
        if path_list[-1] == '':
            path_list = path_list[:-1]
        
        return path_list        
        

#### Test Cases

In [8]:
# Here are some test cases and expected outputs you can use to test your implementation

print('Test case 1')


# create the router and add a route
router = Router("root handler", "not found handler") # remove the 'not found handler' if you did not implement this
router.add_handler("/home/about", "about handler")  # add a route

# some lookups with the expected output
print(router.lookup("/")) # should print 'root handler'
print(router.lookup("/home")) # should print 'not found handler' or None if you did not implement one
print(router.lookup("/home/about")) # should print 'about handler'
print(router.lookup("/home/about/")) # should print 'about handler' or None if you did not handle trailing slashes
print(router.lookup("/home/about/me")) # should print 'not found handler' or None if you did not implement one
print()


print('Test case 2')
# should print 'root handler'
# root handler ('root handler'): You Got Root!
print("\nroot handler ('root handler'):", router.lookup("/"))

# should print 'not found handler' or None if you did not implement one
# /home ('not found handler' or None): HTTP 404 Page Not Found
print("/home ('not found handler' or None):", router.lookup("/home"))

# should print 'about handler'
# /home/about ('about handler'): about handler
print("/home/about ('about handler'):", router.lookup("/home/about"))

# should print 'about handler' or None if you did not handle trailing slashes
# /home/about/ ('about handler' or None): about handler
print("/home/about/ ('about handler' or None):", router.lookup("/home/about/"))

# should print 'not found handler' or None if you did not implement one
# /home/about/me ('not found handler' or None): HTTP 404 Page Not Found
print("/home/about/me ('not found handler' or None):",
      router.lookup("/home/about/me"))

# /home/contact ('contact us'): contact us
print("/home/contact ('contact us'):", router.lookup("/home/contact/"))
print()

print('Test 3')
rout1 = Router("handle this", "")
rout1.add_handler("/something", "something handler")
rout1.add_handler("/something/somewhere", "somewhere handler")

# /home (not found): HTTP 404 Page Not Found
print("\n/home (not found):", rout1.lookup("/home"))

# /something ('something handler'): something handler
print("/something ('something handler'):", rout1.lookup("/something/"))

# /something/somewhere ('somewhere handler'): somewhere handler
print("/something/somewhere ('somewhere handler'):",
      rout1.lookup("/something/somewhere"))
print()


# Reference: https://github.com/ssi112/data-structures-algorithms/blob/master/project3/7_problem.py
# https://github.com/suhassrivats/Udacity-Data-Structures-and-Algorithms/blob/master/3_ProblemsVsAlgorithms/problem_7_RequestRoutingInAWebServerWithATrie.py

Test case 1
You Got Root!
HTTP 404 Page Not Found
about handler
about handler
HTTP 404 Page Not Found

Test case 2

root handler ('root handler'): You Got Root!
/home ('not found handler' or None): HTTP 404 Page Not Found
/home/about ('about handler'): about handler
/home/about/ ('about handler' or None): about handler
/home/about/me ('not found handler' or None): HTTP 404 Page Not Found
/home/contact ('contact us'): HTTP 404 Page Not Found

Test 3

/home (not found): HTTP 404 Page Not Found
/something ('something handler'): something handler
/something/somewhere ('somewhere handler'): somewhere handler

